**[Machine Learning Explainability Home Page](https://www.kaggle.com/learn/machine-learning-explainability)**

---


## イントロ

あなたは[Taxi Fare Prediction](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction)のコンペをやるに当たって、順番の重要性について考えたり計算したりするだろう。ここでは、データ探索や、モデル構築については議論しない。

あなたは下のセルを走らせるだけでOKだ。
- データの読み込み
- データを訓練データとテストデータに分類
- タクシーの運賃を予測するモデル構築
- 数行を確認のために記述

In [ ]:
# データを読み込み、分類、モデル構築とEDAを行うためのライブラリーインポート
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

data = pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv', nrows=50000)

# 極端な外れ値の座標または負の運賃のデータを削除する
data = data.query('pickup_latitude > 40.7 and pickup_latitude < 40.8 and ' +
                  'dropoff_latitude > 40.7 and dropoff_latitude < 40.8 and ' +
                  'pickup_longitude > -74 and pickup_longitude < -73.9 and ' +
                  'dropoff_longitude > -74 and dropoff_longitude < -73.9 and ' +
                  'fare_amount > 0'
                  )

y = data.fare_amount

base_features = ['pickup_longitude',
                 'pickup_latitude',
                 'dropoff_longitude',
                 'dropoff_latitude',
                 'passenger_count']

X = data[base_features]


train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
first_model = RandomForestRegressor(n_estimators=50, random_state=1).fit(train_X, train_y)

# フィードバックシステムのための環境セットアップ
from learntools.core import binder
binder.bind(globals())
from learntools.ml_explainability.ex2 import *
print("Setup Complete")

# データを見る
print("Data sample:")
data.head()

次の2つのセルは、トレーニングデータの値を理解するのにも役立ちます。

In [ ]:
train_X.describe()

In [ ]:
train_y.describe()

## 質問1

最初のモデルは次の機能を使用します
- pickup_longitude
- pickup_latitude
- dropoff_longitude
- dropoff_latitude
- passenger_count

コードを実行する前に、タクシー料金を予測するために潜在的に役立つと思われる変数はどれですか？順列の重要性によって、これらの機能が重要であると必ずしも識別されると思いますか？

それについて考えたら、以下の `q_1.solution（）`を実行して、コードを実行する前にこれについてどう考えるかを確認してください。

In [ ]:
# あなたの答えを確認しよう (このコードを走らせてみてください！)
q_1.solution()

## 質問2

`perm`という`PermutationImportance`オブジェクトを作成して、`first_model`の重要性を示します。適切なデータでモデルを適用し、重みづけを算出しましょう。便宜上、チュートリアルのコードはこのコードセルのコメントにコピーされています。

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

# この問題で使用するには、以下のコードを少し変更してください。 
#perm = PermutationImportance(my_model, random_state=1).fit(val_X, val_y)

# あなたの答えを確認しよう
q_2.check()

# 次の行のコメントアウトを解除して、結果を視覚化します
#eli5.show_weights(perm, feature_names = val_X.columns.tolist())

ヒントまたは解決策を確認するには、以下の行のコメントを解除してください。

In [ ]:
# q_2.hint()
# q_2.solution()

## 質問3
これらの結果を見る前に、4つの方向の特徴のそれぞれが等しく重要であると予想していたかもしれません。
しかし、平均して、緯度の特徴は経度の特徴よりも重要です。これについての仮説を立てることはできますか？
あなたがそれについて考えた後、いくつかの可能な説明のためにここをチェックしてください：

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_3.solution()

## 質問4


ニューヨーク市の詳細な知識がなければ、緯度よりも経度が重要である理由に関するほとんどの仮説を除外することは困難です。次の良いステップは、総移動距離の影響から都市の特定の場所にいる影響を解きほぐすことです。

以下のコードは、縦方向および横方向の距離の新しい機能を作成します。次に、これらの新しい機能を既存の機能に追加するモデルを構築します。

この新しい機能セットを使用して重要度の重みを計算および表示するには、2行のコードを入力します。いつものように、以下の行のコメントを解除して、コードを確認したり、ヒントを参照したり、解決策を取得したりできます。

In [ ]:
# create new features
data['abs_lon_change'] = abs(data.dropoff_longitude - data.pickup_longitude)
data['abs_lat_change'] = abs(data.dropoff_latitude - data.pickup_latitude)

features_2  = ['pickup_longitude',
               'pickup_latitude',
               'dropoff_longitude',
               'dropoff_latitude',
               'abs_lat_change',
               'abs_lon_change']

X = data[features_2]
new_train_X, new_val_X, new_train_y, new_val_y = train_test_split(X, y, random_state=1)
second_model = RandomForestRegressor(n_estimators=30, random_state=1).fit(new_train_X, new_train_y)

# Create a PermutationImportance object on second_model and fit it to new_val_X and new_val_y
# Use a random_state of 1 for reproducible results that match the expected solution.
perm2 = ____

# show the weights for the permutation importance you just calculated
____

# Check your answer
q_4.check()

これらの重要度スコアをどのように解釈しますか？移動距離は、ロケーション効果よりもはるかに重要と思われます。

ただし、場所は依然としてモデルの予測に影響し、ドロップオフの場所はピックアップの場所よりもわずかに重要になります。なぜそうなるのか、仮説はありますか？次のレッスンのテクニックは、これをさらに掘り下げるのに役立ちます。

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_4.solution()

## 質問5

`abs_lon_change`と`abs_lat_change`の値が非常に小さい（すべての値が-0.1〜0.1である）のに、他の変数の値は大きいようです。この場合、これらの座標がより大きな順列重要度値を持つ理由を説明できますか？

これらの機能の100倍の大きさの機能を作成して使用し、その大きな機能をトレーニングと重要度の計算に使用した代替案を検討してください。これにより、出力される順列の重要度の値が変わりますか？

その理由はどうでえしょうか？

答えを考えた後、この実験を試すか、下のセルで答えを調べてください。

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_5.solution()

## 質問6

緯度方向の距離の特徴の重要性は、経度方向の距離の重要性よりも大きいことがわかりました。このことから、固定された緯度方向の距離を移動することは、同じ縦方向の距離を移動するよりも高価になる傾向があるかどうかを結論付けることができますか？

なぜですか？以下の回答を確認してください。

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_6.solution()

## 継続的にやってみよう

順番の重要性は、デバッキング、モデルの理解、モデルから高レベルの概要を伝えるのに役に立ちます。

次は [部分従属プロット](https://www.kaggle.com/dansbecker/partial-plots)でどのようにしたら、特徴量が予測に貢献できるかを調べてみましょう。

---
**[Machine Learning Explainability Home Page](https://www.kaggle.com/learn/machine-learning-explainability)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*